In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Optional
from pathlib import Path
import pandas as pd
from pandas import Series, DataFrame
from dataclasses import dataclass

import pandas_indexing.accessors
from pandas_indexing import isin, ismatch, semijoin, extractlevel

In [34]:
from aneris.harmonize import Harmonizer
from aneris.downscaling import Downscaler

# Read model and historic data including overrides

In [3]:
base_path = Path(
    "/Users/coroa/Library/CloudStorage/OneDrive-SharedLibraries-IIASA/RESCUE - WP 1/data"
)

## Variable definition files

The variable definition file is a CSV or yaml file that needs to contain the `variable`-name, its `sector`, `gas` components and whether it is expected `global` (or regional instead).

Here we generate one based on the cmip6 historical data we have that could be used as a basis but we would want to finetune this by hand.

### Create new variable definition files from historical data

In [4]:
variables = (
    pd.read_csv(
        base_path / "historical/cmip6/history.csv",
        usecols=[3, 4],
    )
    .rename(columns=str.lower)
    .drop_duplicates()
    .set_index("variable")
)

In [5]:
variabledefs = (
    extractlevel(variables, "CEDS+|9+ Sectors|Emissions|{gas}|{sector}|Unharmonized")
    # .loc[lambda df: ~ismatch(df, sector="Aggregate - *")]
    .assign(**{"global": False})
)
variabledefs.loc[
    isin(gas=["F-Gases", "N2O", "C2F6", "HFC", "SF6", "CF4"]), "global"
] = True
variabledefs.loc[
    isin(
        sector=["Aggregate - Agriculture and LUC", "Aircraft", "International Shipping"]
    ),
    "global",
] = True

In [6]:
variabledefs.to_csv("variabledefs.csv")

### Load variable definitions

In [7]:
@dataclass
class VariableDefinitions:
    data: DataFrame

    @classmethod
    def from_csv(cls, path):
        return cls(pd.read_csv("variabledefs.csv", index_col=list(range(3))))

    @property
    def variable_index(self):
        return self.data.index

    @property
    def index_global(self):
        return self.data.index[self.data["global"]].idx.project(["gas", "sector"])

    @property
    def index_regional(self):
        return self.data.index[~self.data["global"]].idx.project(["gas", "sector"])

    def load_data(
        self,
        df: DataFrame,
        levels: Optional[list[str]] = None,
        ignore_undefined: bool = True,
        ignore_missing: bool = False,
        timeseries: bool = True,
    ):
        """Load data from dataframe

        Assigns sector/gas and checks correct units.

        Parameters
        ----------
        df : DataFrame
            data
        levels : list of str, optional
            levels to keep, or all if None
        ignore_undefined : bool, default True
            whether to fail if undefined variables exist in `df`
        ignore_missing : bool, default False
            whether to ignore defined variables missing from `df`
        timeseries : bool, default True
            whether data is a timeseries and columns should be cast to int

        Returns
        -------
        DataFrame
            data with sector/gas index levels

        Note
        ----
        Does not check regional availability yet! Also because it would have to
        understand about regionmappings and aggregations then.
        """

        df = df.rename_axis(index=str.lower)
        if timeseries:
            df = df.rename_axis(columns="year").rename(columns=int)
        else:
            df = df.rename(columns=str.lower)

        if ignore_undefined and ignore_missing:
            how = "inner"
        elif ignore_undefined:
            how = "right"
        else:
            how = "outer"
        index, li, ri = df.index.join(
            self.variable_index, how=how, return_indexers=True
        )

        def unique_variable_str(index):
            return "\n  " + ",\n  ".join(index.unique("variable"))

        if (li == -1).any():
            raise ValueError(
                "Variables missing from data:" + unique_variable_str(index[li == -1])
            )
        if (ri == -1).any():
            raise ValueError(
                "Undefined variables exist in data:"
                + unique_variable_str(index[ri == -1])
            )

        df = pd.DataFrame(df.values[li], index=index, columns=df.columns).__finalize__(
            df
        )
        if timeseries:
            data_units = self.data["unit"].values[ri]
            non_matching_units = df.index.idx.project("unit") != data_units
            if non_matching_units.any():
                errors = (
                    df.index.to_frame(index=False)
                    .loc[non_matching_units, ["model", "scenario", "variable", "unit"]]
                    .assign(**{"expected unit": data_units[non_matching_units]})
                    .drop_duplicates()
                )
                raise ValueError(
                    "Some variables in the data do not have the correct units:\n"
                    + errors.to_string(index=False)
                )

        if levels is not None:
            return df.idx.project(levels)
        return df

In [8]:
variabledefs = VariableDefinitions.from_csv("variabledefs.csv")

## RegionMapping helps reading in a region definition file

In [9]:
@dataclass
class RegionMapping:
    data: pd.Series

    @classmethod
    def concat(cls, rms):
        return cls(pd.concat(rm.data for rm in rms))

    @classmethod
    def from_regiondef(cls, path):
        path = Path(path)
        match path.suffix:
            case ".csv":
                df = pd.read_csv(path)
            case ".xlsx":
                df = pd.read_csv(path)
            case suffix:
                raise ValueError(f"Unknown file suffix: {suffix}")

        return cls(
            df.set_index("ISO Code")["Native Region Code"]
            .rename_axis("country")
            .rename(index=str.lower)
            .rename("region")
        )

    def prefix(self, s: str):
        return self.__class__(s + self.data)

    @property
    def index(self) -> pd.MultiIndex:
        return pd.MultiIndex.from_arrays(
            [self.data.index, self.data.values], names=["country", "region"]
        )

    def aggregate(self, df: DataFrame, level="country") -> DataFrame:
        if level != "country":
            df = df.rename_axis(index={level: "country"})
        return (
            df.idx.semijoin(self.index, how="right")
            .groupby(
                [n if n != "country" else "region" for n in df.index.names],
                dropna=False,
            )
            .sum()
        )

In [10]:
regionmapping = RegionMapping.from_regiondef(
    base_path / "historical/cmip6/remind_region_mapping.csv"
)

## Model and historic data read in

Can be read in and prepared using `read_iamc` or the `variabledefs`

In [11]:
hist = pd.read_csv(
    base_path / "historical/cmip6/history.csv", index_col=list(range(5))
).pipe(variabledefs.load_data, levels=["region", "gas", "sector", "unit"])

In [12]:
model = (
    pd.read_excel(
        base_path / "iam_files/cmip6/REMIND-MAGPIE_SSP5-34-OS/ssp5-34-os.xlsx",
        sheet_name="data",
        index_col=list(range(5)),
    )
    .rename(index={"Mt CO2-equiv/yr": "Mt CO2-eq/yr"}, level="Unit")
    .pipe(
        variabledefs.load_data,
        levels=["model", "scenario", "region", "gas", "sector", "unit"],
        ignore_missing=True,
    )
)

In [13]:
harm_overrides = (
    pd.read_excel(
        base_path / "iam_files/cmip6/REMIND-MAGPIE_SSP5-34-OS/ssp5-34-os.xlsx",
        sheet_name="harmonization",
        index_col=list(range(4)),
        usecols=list(range(5)),
    )
    .rename_axis(index=str.lower)
    .rename(columns=str.lower)
    .pipe(
        variabledefs.load_data,
        ignore_missing=True,
        levels=["region", "gas", "sector"],
        timeseries=False,
    )
    .method
)

harm_overrides.head()

region  gas  sector        
CHN     BC   Energy Sector     reduce_offset_2150
AFR     BC   Forest Burning        constant_ratio
EUR     BC   Forest Burning        constant_ratio
ROW     BC   Forest Burning        constant_ratio
USA     BC   Forest Burning        constant_ratio
Name: method, dtype: object

In [14]:
model.head()

year                                                                           2005   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0439  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1055   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0387   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0817   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2010   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0633  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1164   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0384   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0841   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2015   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0799  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1243   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0892   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2020   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0966  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1322   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0943   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2030   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1279  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1400   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0369   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1054   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2040   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1568  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1509   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0357   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1061   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2050   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1768  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.

# Harmonization

## Preparation of input data

In [15]:
hist_agg = pd.concat(
    [
        hist.idx.semijoin(variabledefs.index_regional, how="inner").pipe(
            regionmapping.aggregate, level="region"
        ),
        hist.idx.semijoin(variabledefs.index_global, how="inner").loc[
            isin(region="World")
        ],
    ]
)

In [16]:
model_agg = pd.concat(
    [
        model.idx.semijoin(variabledefs.index_regional, how="inner").loc[
            isin(region=regionmapping.data.unique())
        ],
        model.idx.semijoin(variabledefs.index_global, how="inner").loc[
            isin(region="World")
        ],
    ]
)

## Harmonize all model, scenarios combinations

In [27]:
year = 2015

In [28]:
harmonized = []
for m, s in model.index.idx.project(["model", "scenario"]).unique():
    scen = model_agg.loc[isin(model=m, scenario=s)].droplevel(["model", "scenario"])
    h = Harmonizer(
        scen, hist_agg.idx.semijoin(scen.index, how="right"), harm_idx=scen.index.names
    )
    result = h.harmonize(year=year, overrides=harm_overrides)
    harmonized.append(result.idx.assign(model=m, scenario=s))
harmonized = pd.concat(harmonized)

# TODO harmonization casts columns to str!!
harmonized = harmonized.rename(columns=int)

/Users/coroa/.local/conda/envs/concordia/lib/python3.11/site-packages/aneris/methods.py:389: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.std(x) / np.mean(x))
INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with model_zero
INFO:root:Harmonizing with hist_zero
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce_offset_2150
INFO:root:Harmonizing with reduce_ratio_2080
INFO:root:Harmonizing with constant_offset
INFO:root:Harmonizing with reduce_offset_2100


In [29]:
harmonized.head()

year                                                                             2015   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.058867  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.013035   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.004719   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.005197   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2020   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.076346  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.025056   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.005963   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.013408   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2030   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.109204  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.041098   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.007051   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.030730   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2040   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.139662  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.060239   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.008338   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.037653   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2050   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.161220  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.094981   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.009125   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.040575   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2060   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.146278  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.085123   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.011613   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.047498   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2070   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0

# Downscaling

In [35]:
gdp = (
    pd.read_csv(
        base_path / "historical" / "SspDb_country_data_2013-06-12.csv",
        index_col=list(range(5)),
    )
    .rename_axis(index=str.lower)
    .loc[
        isin(
            model="OECD Env-Growth",
            scenario=[f"SSP{n+1}_v9_130325" for n in range(5)],
            variable="GDP|PPP",
        )
    ]
    .dropna(how="all", axis=1)
    .rename_axis(index={"scenario": "ssp", "region": "country"})
    .rename(index=str.lower, level="country")
    .rename(columns=int)
    .idx.project(["ssp", "country"])
)
gdp.head()

2000    2005     2010     2015      2020      2025   
ssp            country                                                         
SSP1_v9_130325 bhs       9.179   9.964    9.647   10.852    12.364    13.913  \
               bih      18.823  23.975   28.065   30.809    37.787    47.037   
               blr      58.129  83.492  118.671  145.259   176.778   211.172   
               blz       1.404   1.825    2.061    2.266     2.592     3.141   
               twn       0.000   0.000  742.813  889.439  1073.988  1208.735   

                            2030      2035      2040      2045  ...      2055   
ssp            country                                          ...             
SSP1_v9_130325 bhs        15.577    17.400    19.244    21.071  ...    24.061  \
               bih        57.740    69.273    80.304    89.614  ...   101.533   
               blr       250.038   292.074   330.894   362.482  ...   404.928   
               blz         3.903     4.861     5.977     7.218  ...     9.869   
               twn      1321.155  1424.861  1513.084  1570.705  ...  1587.286   

                            2060      2065      2070      2075      2080   
ssp            country                                                     
SSP1_v9_130325 bhs        25.282    26.374    27.331    27.979    28.462  \
               bih       105.790   108.039   109.080   108.896   107.453   
               blr       424.762   443.176   455.985   461.441   462.309   
               blz        11.223    12.574    13.881    15.116    16.320   
               twn      1574.961  1608.617  1631.834  1638.589  1633.208   

                            2085      2090      2095      2100  
ssp            country                                          
SSP1_v9_130325 bhs        28.752    28.843    28.773    28.526  
               bih       105.156   102.274    99.093    95.538  
               blr       461.472   459.908   456.940   451.145  
               blz        17.463    18.516    19.481    20.309  
               twn      1610.471  1576.924  1534.288  1482.390  

[5 rows x 21 columns]

In [36]:
SSP_per_pathway = (
    harmonized.index.idx.project(["model", "scenario"])
    .unique()
    .to_frame()
    .scenario.str.extract("(SSP[1-5])")[0]
    .fillna("SSP2")
)
gdp = semijoin(
    gdp,
    SSP_per_pathway.index.idx.assign(ssp=SSP_per_pathway + "_v9_130325"),
    how="right",
).idx.project(["model", "scenario", "country"])

In [37]:
harmonized = harmonized.idx.semijoin(variabledefs.index_regional, how="inner").loc[isin(region="AFR", gas=["CH4", "CO2"])] # sector=["Waste", "Energy Sector"], gas=["CH4", "CO2"])]

In [38]:
hist = hist.idx.semijoin(variabledefs.index_regional, how="inner").rename_axis(
        index={"region": "country"}
).loc[isin(gas=["CH4", "CO2"])]

In [39]:
downscaler = Downscaler(harmonized, hist, 2015, regionmapping.data.drop(["srb (kosovo)", "ssd"]), gdp=gdp)

In [42]:
from aneris.downscaling import Downscaler

downscaler = Downscaler(
    harmonized.idx.semijoin(variabledefs.index_regional, how="inner"),
    hist.idx.semijoin(variabledefs.index_regional, how="inner").rename_axis(
        index={"region": "country"}
    ),
    year,
    regionmapping.data.drop(["srb (kosovo)", "ssd"]),
    gdp=gdp,
)
results = downscaler.downscale()

/Users/coroa/.local/conda/envs/concordia/lib/python3.11/site-packages/aneris/methods.py:389: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.std(x) / np.mean(x))


In [43]:
results

year                                                                                               2015   
gas sector                              region unit      model         scenario       country             
CH4 Agriculture                         AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 ago      0.220823  \
                                                                                      ben      0.095516   
                                                                                      bwa      0.105129   
                                                                                      bfa      0.511754   
                                                                                      bdi      0.036465   
...                                                                                                 ...   
    Solvents Production and Application AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 zwe      0.000000   
                                                                                      ago      0.000000   
                                                                                      bdi      0.000000   
                                                                                      ben      0.000000   
                                                                                      bfa      0.000000   

year                                                                                               2020   
gas sector                              region unit      model         scenario       country             
CH4 Agriculture                         AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 ago      0.279381  \
                                                                                      ben      0.120845   
                                                                                      bwa      0.133007   
                                                                                      bfa      0.647461   
                                                                                      bdi      0.046135   
...                                                                                                 ...   
    Solvents Production and Application AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 zwe      0.000000   
                                                                                      ago      0.000000   
                                                                                      bdi      0.000000   
                                                                                      ben      0.000000   
                                                                                      bfa      0.000000   

year                                                                                               2030   
gas sector                              region unit      model         scenario       country             
CH4 Agriculture                         AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 ago      0.515045  \
                                                                                      ben      0.222780   
                                                                                      bwa      0.245201   
                                                                                      bfa      1.193611   
                                                                                      bdi      0.085051   
...                                                                                                 ...   
    Solvents Production and Application AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 zwe      0.000000   
                                                                                      ago      0.000000   
                                                                                      bdi      0.000000   
                                       